# Libraries

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
import sys
sys.path.append('/content/gdrive/MyDrive/Pythorch/ArcFace/ArcFace')

In [3]:
import os
from pathlib import Path

from tqdm import tqdm
from easydict import EasyDict as edict

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as vutils
from torchvision import transforms as trans


from data.ms1m import get_train_loader
from data.lfw import LFW

from backbone.arcfacenet import SEResNet_IR
from margin.ArcMarginProduct import ArcMarginProduct

from util.utils import save_checkpoint , test


# Configuration

In [4]:
conf = edict()

conf.train_root = '/content/gdrive/MyDrive/Pythorch/ArcFace/ArcFace/dataset/MS1M'
conf.lfw_test_root = '/content/gdrive/MyDrive/Pythorch/ArcFace/ArcFace/dataset/lfw_aligned_112'
conf.lfw_file_list = '/content/gdrive/MyDrive/Pythorch/ArcFace/ArcFace/dataset/lfw_pair.txt'

conf.mode = 'se_ir' # We could also use 'ir' model.
conf.depth = 50 # 100 , 152
conf.margin_type = 'ArcFace'
conf.feature_dim = 512
conf.scale_siz = 32.0
conf.batch_size = 96 # 16 , 32 , 48 ,...
conf.lr = 0.01
conf.milestones = [8,10,12]
conf.total_epoch = 14

conf.save_folder = '/content/gdrive/MyDrive/Pythorch/ArcFace/ArcFace/saved'
conf.save_dir = os.path.join(conf.save_folder , conf.mode + '_' + str(conf.depth)) # ./save/se_ir_152
conf.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
conf.num_workers = 4 # cpu workers num
conf.pin_memory = True


In [5]:
os.makedirs(conf.save_dir , exist_ok = True)

# Dataset Load

In [6]:
transform = trans.Compose([
    trans.ToTensor() , # range(0,255) --> [0.0 , 1.0]
    trans.Normalize(mean = (0.5,0.5,0.5) , std = (0.5,0.5,0.5)) # We give 3 value for all values R-G-B.
])

trainloader , class_num = get_train_loader(conf)

In [7]:
print('number of id : ' , class_num)

number of id :  200


In [8]:
print(trainloader.dataset)

Dataset ImageFolder
    Number of datapoints: 29148
    Root location: /content/gdrive/MyDrive/Pythorch/ArcFace/ArcFace/dataset/MS1M
    StandardTransform
Transform: Compose(
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
           )


In [9]:
lfwdataset = LFW(conf.lfw_test_root , conf.lfw_file_list , transform = transform)
lfwloader = torch.utils.data.DataLoader(lfwdataset , batch_size = 128 , num_workers = conf.num_workers)

# Model

In [10]:
print(conf.device)

cuda:0


In [11]:
net = SEResNet_IR(num_layers = conf.depth , feature_dim = conf.feature_dim , mode = conf.mode ).to(conf.device)
margin = ArcMarginProduct(conf.feature_dim , class_num).to(conf.device)

In [12]:
print(net)

SEResNet_IR(
  (input_layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=64)
  )
  (output_layer): Sequential(
    (0): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=0.4, inplace=False)
    (2): Flatten()
    (3): Linear(in_features=25088, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (body): Sequential(
    (0): BottleNeck_IR_SE(
      (shortcut_layer): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
      (res_layer): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (2): BatchNorm2d(64, eps=1e-05, moment

In [13]:
criterion = nn.CrossEntropyLoss()

In [14]:
optimizer = optim.SGD([
    {'params' : net.parameters() ,'weight_decay' : 5e-4 },
    {'params' : margin.parameters() , 'weight_decay' : 5e-4}
] , lr = conf.lr , momentum = 0.9 , nesterov = True , )

In [15]:
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.01
    maximize: False
    momentum: 0.9
    nesterov: True
    weight_decay: 0.0005

Parameter Group 1
    dampening: 0
    foreach: None
    lr: 0.01
    maximize: False
    momentum: 0.9
    nesterov: True
    weight_decay: 0.0005
)


In [ ]:
# class Model(torch.nn.Module):
#     def __init__(self, model, pool):
#         super(Model, self).__init__()

In [ ]:
# class Skip(nn.Module):
#     def forward(self, x):
#         return x

# class SkipEdge(Edge):
#     def __init__(self):
#         super().__init__()
#         self.f = Skip()

In [22]:
optimizer.parameters() # I'll fix it later...

AttributeError: ignored

In [16]:
def schedule_lr():
  for params in optimizer.parameters():
    params['lr'] /= 10
  print(optimizer)

# Train

In [17]:
best_acc = 0

for epoch in range(1,conf.total_epoch+1):

  net.train()

  #net.eval()

  print(f'Epoch : {epoch}/{conf.total_epoch}')
  
  if epoch in conf.milestones:
    schedule_lr()
  # if epoch == conf.milestones[0]:
  #   schedule_lr()
  # if epoch == conf.milestones[1]:
  #   schedule_lr()
  # if epoch == conf.milestones[2]:
  #   schedule_lr()

Epoch : 1/14
Epoch : 2/14
Epoch : 3/14
Epoch : 4/14
Epoch : 5/14
Epoch : 6/14
Epoch : 7/14
Epoch : 8/14


AttributeError: ignored